<a href="https://colab.research.google.com/github/glauciodrumond/OracleCloud/blob/main/Projeto_Cloud_Networking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PROJETO CLOUD IMPLEMENTAÇÃO NETWORKING - OCI**

IMPLEMENTAÇÃO DE CONTROLE DE COMUNICAÇÃO DE REDE NA ORACLE CLOUD INFRASTRUCTURE(OCI) COM VIRTUAL CLOUD NETWORK(VCN), SUBREDES DISTINTAS E SECURITY LISTS COM REGRAS ESPECÍFICAS PARA CADA SUBREDE.

* **Projeto realizado no curso da:**<br>
[<img src="https://thecloudbootcamp.com/wp-content/uploads/2020/09/The-Cloud-Bootcamp-big-transparent-1024x1024.png.webp" alt="drawing" width="400" align="center"/>](https://thecloudbootcamp.com/)
<br><img src="https://miro.medium.com/max/800/0*-nyd8VIUz-CqGR5B.png" alt="drawing" width="600" align="center"/>


# Contextualização


![](https://drive.google.com/uc?export=view&id=1MDEWOZi4JwOUl5jUBy7mh9IY9hteNz1y)   

<br>Imagine que você está trabalhando em um projeto de Oracle Cloud Infrastructure como Especialista em Cloud. Dentro da sua empresa,o seu Gestor de Cloud compartilhou com você a demanda de implementar o Global Financing Portal que tem como objetivo permitir que as pessoas acessem e façam, por exemplo, solicitações de reembolso. 

<br>Conversando com o seu Gestor, ele compartilhou com você pontos relacionados a elevação do nível de Networking da empresa. Você um conhecedor das melhores práticas, definiu os pré-requisitos:

* Implementar a segmentação de rede
    * Global Financing Portal > Subrede Financeiro
    * Estações de Trabalho Virtual > Subrede Comercial

* Utilizar os recursos da Security List, para controlar a comunicação de rede definindo regras bem específicas para cada Subrede.

<br>Agora, chegou a hora de você, Especialista em Cloud, colocar a mão na massa para implementar essa demanda incrível.


## Pré-requisitos

Link para download do Gitbash e VNC

* Gitbash:

  * https://git-scm.com/downloads

* VNC Viewer:

  * https://www.realvnc.com/en/connect/download/viewer/



# Solução do Problema

## Criação dos recursos de rede

**Na console da OCI, acessar o serviço Virtual Cloud Networks**

### **Certificar-se de estar no compartment para a criação dos recursos de Rede**
* Clicar em **Create VCN** preenchendo com as informações abaixo:
  * Name: vcn-mod3
  * CIDR Block: 10.0.0.0/16
  * Clicar em Create VCN

### **Criar subrede-financeiro**

Acessar a **VCN criada**, em **Subnets**, certificar-se de estar no compartment para a criação dos recursos de rede
* Clicar em **Create Subnet** preenchendo com as informações abaixo:
  * Name: subrede-financeiro
  * CIDR Block: 10.0.1.0/24
  * Clicar em Create Subnet

### **Criar subrede-comercial**

* Ainda em Subnets, clicar em **Create Subnet** preenchendo com as informações abaixo:
  * Name: subrede-comercial
  * CIDR Block: 10.0.2.0/24
  * Clicar em Create Subnet

## Criar Security List: **sl-comercial** e **sl-financeiro**

### **sl-comercial**

* **Ingress:**

  * Source: Inserir o Seu IP

  * Exemplo: 50.89.141.235/32

  * Dica: Site para verificar o Seu IP: https://www.meuip.com.br/

  * Destination Port: 22

* **Egress:**

  * Source: 0.0.0.0/0

  * Destination Port: All protocol/ports


### **sl-financeiro**

* **Primeira Ingress rule:**

  * Source: Inserir o Seu IP

  * Exemplo: 50.89.141.235/32

  * Dica: Site para verificar o Seu IP: https://www.meuip.com.br/

* **Segunda Ingress Rule:**

  * Destination Port: 22

  * Source: 10.0.2.0/24

  * Destination Port: 80

* **Egress:**

  * Source: 0.0.0.0/0

  * Destination Port: All protocol/ports

### **Associar Security Lists às Subredes**

  * sl-comercial -> subrede-comercial

  * sl-financeiro -> subrede-financeiro

  * **Remover** Default Security List de subrede-comercial e subrede-financeiro

## **Criar Internet Gateway**

* Na VCN criada, em Internet Gateways, certificar-se de estar no compartment para a criação dos recursos de rede
* Clicar em **Create Internet Gateway** preenchendo com as informações abaixo:
  * Name: internet-gw-mod3
  * CIDR Block: 10.0.1.0/24
  * Clicar em Create Subnet

* Adicionar rota com destino ao Internet Gateway
  * Target Type: Internet Gateway
  * Destination CIDR Block: 0.0.0.0/0
  * Target Internet Gateway: internet-gw-mod3
  * Clicar em Add Route Rules

## Criação da Instância do Financeiro
**Na console da OCI, acessar o serviço Instances e executar os passos abaixo:**
* Certificar-se de estar no compartment para a criação dos recursos de Compute
* Clicar em **Create instance**, preenchendo com as informações abaixo:
  * **Name:** vm-financeiro
  * **Shape/OS:** Always Free / Oracle Linux 7 ( Atenção: Não selecione Oracle Linux 8, o procedimento foi validado no Oracle Linux 7)
  * Em **Networking**, clicar em **Edit**
    * Na seção **Network**, selecionar **Select existing virtual cloud network**
    * Selecionar o compartment destinado para a criação dos recursos de Rede
    * Certificar-se da VCN **vcn-mod3** estar selecionada
    * Na seção **Subnet**, selecionar **Select existing virtual cloud network**
    * Selecionar o compartment destinado para a criação dos recursos de Rede
    * Selecionar a subrede **subrede-financeiro**
  * Em **Add SSH Keys, Generate SSH key pair**, clicar em **Save Private Key**. ( Fique alerta ao local que salvou a chave privada, pois utilizaremos ela nos próximos passos. )
  * Clicar em **Create**.

<br>**Utilizando o Gitbash, executar o comando abaixo para acessar via SSH a instância vm-financeiro**
<br>`ssh -i <caminho-chave-privada-financeiro> opc@<ip-público-da-vm-financeiro>`

<br>**Concluído o acesso à instância, executar os comandos abaixo:**

```
sudo yum install httpd -y
sudo apachectl start
sudo systemctl enable httpd
sudo firewall-cmd --zone=public --add-service=http
sudo firewall-cmd --permanent --zone=public --add-service=http
cd /var/www/html/
sudo wget https://objectstorage.us-ashburn-1.oraclecloud.com/p/8pVpr-wTqCWT7xuMdxWu9z5pSD5UjCYpXFaInFrhW3tmoySyD2eR2Foz6ksQWm5y/n/idqfa2z2mift/b/bootcamp-oci/o/oci-f-handson-modulo-networking-website-files.zip
sudo unzip oci-f-handson-modulo-networking-website-files.zip
sudo chown apache:apache *.*
sudo rm -rf __MACOSX oci-f-handson-modulo-networking-website-files.zip

```

## Criação da Instância do Comercial
**Na console da OCI, acessar o serviço Instances e executar os passos abaixo:**
* Certificar-se de estar no compartment para a criação dos recursos de Compute
* Clicar em **Create instance**, preenchendo com as informações abaixo:
  * **Name:** vm-comercial
  * **Shape/OS:** Always Free / Oracle Linux 7 ( Atenção: Não selecione Oracle Linux 8, o procedimento foi validado no Oracle Linux 7)
  * Em **Networking**, clicar em **Edit**
    * Na seção **Network**, selecionar **Select existing virtual cloud network**
    * Selecionar o compartment destinado para a criação dos recursos de Rede
    * Certificar-se da VCN **vcn-mod3** estar selecionada
    * Na seção **Subnet**, selecionar **Select existing virtual cloud network**
    * Selecionar o compartment destinado para a criação dos recursos de Rede
    * Selecionar a subrede **subrede-comercial**
  * Em **Add SSH Keys, Generate SSH key pair**, clicar em **Save Private Key**. ( Fique alerta ao local que salvou a chave privada, pois utilizaremos ela nos próximos passos. )
  * Clicar em **Create**.

<br>**Utilizando o Gitbash, executar o comando abaixo para acessar via SSH a instância vm-comercial**
<br>`ssh -i <caminho-chave-privada-comercial> opc@<ip-público-da-vm-comercial>`

<br>**Concluído o acesso à instância, executar os comandos abaixo para instalar os pacotes GUI e VNC Server:**

```
sudo  yum -y groups install "Server with GUI"
sudo yum group list
sudo yum install tigervnc-server -y

```
<br> **Executar os comandos abaixo para configurar VNC**

```
cd /lib/systemd/system/
sudo cp vncserver@.service /etc/systemd/system/vncserver@:1.service
sudo vi /etc/systemd/system/vncserver@:1.service

```
<br> **Alterar campo USER para opc:**
```
DE:
ExecStart=/usr/bin/vncserver_wrapper <USER> %
PARA:
ExecStart=/usr/bin/vncserver_wrapper opc %
```
<br> **Executar o comando abaixo para criar senha para Acesso VNC:**
```
vncpasswd
Senha: welcome1
Em Would you like to enter a view-only? (y/n)? Insira N

```

<br> **Executar os comandos abaixo para configurar e subir VNC**
```
sudo systemctl enable vncserver@:1.service
sudo systemctl daemon-reload
sudo systemctl start vncserver@:1.service
sudo systemctl status vncserver@:1.service

```
<br> **No Gitbash, executar o comando para iniciar o Tunnel SSH**

```
ssh -L 5901:localhost:5901 opc@<ip-publico-da-vm-comercial> -i <chave-privada-vm-comercial>

```

<br> **Acessar instância via VNC**

* Inserir a senha definida para acesso ao VNC

  * Na vm-comercial, abrir Firefox e testar o acesso ao portal pelo IP Privado da vm-financeiro via porta 80

  * Testar acesso via SSH ao IP privado da vm-financeiro (deve ser negado)





## Resultado


![](https://drive.google.com/uc?export=view&id=1-2yv2rOQ1AJ37-q_e9JDwIdUaZg_x5t-)   

![](https://drive.google.com/uc?export=view&id=1ZUKXPfhsA1j-cNQrMnz7YQ0CcVnAr86K)   



# Referências

https://docs.oracle.com/en/operating-systems/oracle-linux/7/network/ol7-websvc.html#ol7-about-websvc